In [1]:
import numpy as np
import pandas as pd
import ast

# Generic Functions

In [2]:
def get_training_hyper(row, hyper="seed"):
    hypers = ast.literal_eval(row["config/training"])
    return hypers[hyper]

def get_d_hypers(row, hyper="n_hidden_units"):
    hypers = ast.literal_eval(row["config/discriminator"])
    return hypers[hyper] 

def get_g_hypers(row, hyper="n_hidden_units"):
    hypers = ast.literal_eval(row["config/generator"])
    return hypers[hyper] 

def get_expanded_df(results_df, optim="sgd", separate_momentum=False):
    
    # List the training hyperparameters
    if optim == "sgd":
        if separate_momentum:
            training_hypers = ["seed", "d_lr", "g_lr", "gamma", "d_momentum", "g_momentum", "step_size"]
        else:
            training_hypers = ["seed", "d_lr", "g_lr", "gamma", "momentum", "step_size"]
        precise_cols = training_hypers[1:5]
    elif optim == "adam":
        training_hypers = ["seed", "d_lr", "g_lr", "gamma", "g_betas", "d_betas", "step_size"]
        precise_cols = training_hypers[1:4]
    else:
        raise Exception(f"optimizer {optim} not recognized")

    # List the discriminator and generator network hyperparameters
    nn_hypers = ["n_hidden_units", "n_hidden_layers"]

    # Add a new column for each training hyperparameter
    for hyper in training_hypers:
        results_df[hyper] = results_df.apply(lambda row: get_training_hyper(row, hyper), axis=1)
        
    # Extract beta parameters if Adam optimizer is used
    if optim == "adam":
        results_df["g_betas"] = results_df["g_betas"].apply(lambda row: row.strip('][').split())
        results_df["d_betas"] = results_df["d_betas"].apply(lambda row: row.strip('][').split())

    # Add a new column for each discriminator and generator hyperparameter
    for hyper in nn_hypers:
        results_df["d_"+hyper] = results_df.apply(lambda row: get_d_hypers(row, hyper), axis=1)
        results_df["g_"+hyper] = results_df.apply(lambda row: get_g_hypers(row, hyper), axis=1)
        
    # Increase precision of several hypers
    for col in precise_cols:
        results_df[col] = results_df[col].map(lambda x: '{0:.17}'.format(x))
        
    # Drop unnecessary columns
    results_df = results_df.drop(["time_this_iter_s", "done", "timesteps_total", "episodes_total", 
                                  "training_iteration", "experiment_id", "timestamp", "time_total_s", 
                                  "pid", "node_ip", "time_since_restore", "timesteps_since_restore", 
                                  "iterations_since_restore", "trial_id", "config/generator", 
                                  "config/discriminator", "config/problem", "config/training"], axis=1)
        
    return results_df

# 1. EXP

## 1.1 Local results

In [3]:
# Read in the EXP results
exp_results = pd.read_csv("local_results/EXP_results.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results)

In [4]:
# Get all rows for the lowest MSE for each random seed
exp_results.loc[exp_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
710,3.626995e-08,2021-06-09_11-57-53,Blake-Laptop,0,0.074482531278764044,0.065553274742166018,0.99522948392358235,0.92972928344580452,6,30,40,2,3
40,3.748609e-11,2021-06-09_08-14-51,Blake-Laptop,1,0.067565080347116002,0.097713325101597276,0.99388554447792987,0.94263825359745412,15,20,30,3,4
888,2.487941e-14,2021-06-09_13-21-46,Blake-Laptop,2,0.083370696294392768,0.073696586887168403,0.99004539022897808,0.94753511915080102,11,40,40,3,2
772,1.317922e-08,2021-06-09_12-18-13,Blake-Laptop,3,0.07667752653808256,0.048350713184813883,0.9918953808674057,0.92719577739014802,18,40,20,4,4
608,3.046888e-11,2021-06-09_11-26-26,Blake-Laptop,4,0.080288443841802254,0.097000892730916541,0.99576109161909121,0.92890193359920936,20,20,30,4,4
925,1.494417e-08,2021-06-09_13-48-15,Blake-Laptop,5,0.074211171920623389,0.080596868079863396,0.99896598746165399,0.95144702835686645,14,20,20,3,2
494,7.789988e-08,2021-06-09_10-47-15,Blake-Laptop,6,0.044028072972538024,0.092102105724566896,0.99479674794899331,0.93208682737507476,10,20,30,4,4
656,2.107480e-07,2021-06-09_11-43-20,Blake-Laptop,7,0.024223397132966496,0.02834874756532663,0.99989710294340861,0.92648914861366705,19,20,40,2,4
839,1.201932e-08,2021-06-09_12-53-42,Blake-Laptop,8,0.086091031521008901,0.095919451297414263,0.99784006608215936,0.94200568286573905,9,30,30,4,3
749,3.705156e-15,2021-06-09_12-09-29,Blake-Laptop,9,0.094798356875477163,0.068751301110191676,0.99790789529677426,0.96121372336567723,15,30,20,4,4


## 1.2 FASRC results

### 1.2.1 SGD

In [5]:
# Read in the EXP results
exp_results = pd.read_csv("fasrc_results/ray_tune_exp_1.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results)

In [6]:
# Get all rows for the lowest MSE for each random seed
exp_results.loc[exp_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
845,3.495241e-11,2021-09-21_13-40-08,holy7c24304.rc.fas.harvard.edu,0,0.044741834781648067,0.034642778351689524,0.9973765729395736,0.94894585646568241,2,50,40,3,5
1236,6.488770e-10,2021-09-21_13-42-29,holy7c24304.rc.fas.harvard.edu,1,0.053017597673407642,0.064435042053238564,0.99440910661061355,0.91266797764767948,16,50,40,3,3
748,2.015433e-09,2021-09-21_13-39-33,holy7c24304.rc.fas.harvard.edu,2,0.087286036106066847,0.084816773627425096,0.9930350609264248,0.95822415413730866,15,40,30,3,5
1751,1.978099e-11,2021-09-21_13-45-38,holy7c24304.rc.fas.harvard.edu,3,0.081226195987763455,0.092659001121731727,0.99393890741178159,0.96739900189754235,12,50,50,2,4
106,3.422975e-11,2021-09-21_13-35-50,holy7c24304.rc.fas.harvard.edu,4,0.038128188784836661,0.058231769096831656,0.99810402541192433,0.95984204590022038,15,30,50,2,5
480,6.893061e-09,2021-09-21_13-38-30,holy7c24304.rc.fas.harvard.edu,5,0.050435645593104976,0.096831766092763519,0.99535076406166845,0.9226506353685795,8,30,30,4,5
41,6.570240e-08,2021-09-21_13-35-39,holy7c24304.rc.fas.harvard.edu,6,0.065474259225663106,0.092528987670959253,0.99287652236518742,0.90770428748548782,14,50,20,4,3
1224,6.902533e-12,2021-09-21_13-42-44,holy7c24304.rc.fas.harvard.edu,7,0.070604554672835468,0.073295492321867162,0.99295600988377852,0.95764542494010185,14,30,20,5,4
83,2.047026e-10,2021-09-21_13-35-35,holy7c24304.rc.fas.harvard.edu,8,0.080023955003709027,0.081019398608093721,0.9914857462421458,0.96377252758380128,15,50,40,5,2
1323,1.898626e-10,2021-09-21_13-43-10,holy7c24304.rc.fas.harvard.edu,9,0.089586717502232086,0.079339969556743425,0.99896848327714594,0.96288609424133342,12,50,50,2,5


### 1.2.1 Adam

In [22]:
# Read in the EXP results
exp_results = pd.read_csv("fasrc_results/ray_tune_exp_adam_0.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results, optim="adam")

In [23]:
# Get all rows for the lowest MSE for each random seed
exp_results.loc[exp_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
47,3.182350e-16,2021-10-03_10-12-24,holy7c24201.rc.fas.harvard.edu,0,0.012215513377902448,0.094654927836376718,0.9500302442628914,"[0.4913795, 0.31989269]","[0.54223041, 0.26406118]",3,20,40,4,2


In [24]:
exp_results = pd.read_csv("fasrc_results/ray_tune_exp_adam.csv", usecols=np.arange(1,22))
exp_results = get_expanded_df(exp_results, optim="adam")
exp_results.loc[exp_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
344,3.182350e-16,2021-10-04_00-34-42,holy7c24203.rc.fas.harvard.edu,0,0.003463728449223625,0.064090137485700235,0.94530362439311566,"[0.53031704, 0.3746971]","[0.11049184, 0.61258848]",2,40,30,5,5
249,3.182350e-16,2021-10-04_00-34-12,holy7c24203.rc.fas.harvard.edu,1,0.0152970175031675,0.078605929822058052,0.96357947980181735,"[0.42361053, 0.52804964]","[0.26906948, 0.88788478]",3,20,40,3,3
1007,3.182350e-16,2021-10-04_00-39-09,holy7c24203.rc.fas.harvard.edu,2,0.097506579814443814,0.077674898674501128,0.93757903992043412,"[0.65269048, 0.66281278]","[0.40414071, 0.57584149]",3,20,50,4,4
116,3.182350e-16,2021-10-04_00-33-32,holy7c24203.rc.fas.harvard.edu,3,0.035732925640479127,0.060308911115170581,0.90467347451035574,"[0.23153678, 0.83059213]","[0.06683732, 0.26089678]",8,50,50,4,4
18,3.182350e-16,2021-10-04_00-32-46,holy7c24203.rc.fas.harvard.edu,4,0.0098003953476289898,0.033256070589426749,0.94248317439104889,"[0.42579896, 0.89779523]","[0.28879025, 0.71445169]",6,30,50,2,5
175,3.182350e-16,2021-10-04_00-33-53,holy7c24203.rc.fas.harvard.edu,5,0.0067945292193244775,0.055691327344828172,0.93719261819044797,"[0.16960435, 0.61929928]","[0.03860631, 0.84166817]",7,30,40,5,2
118,3.182350e-16,2021-10-04_00-33-34,holy7c24203.rc.fas.harvard.edu,6,0.0096129385952840388,0.07741629065313653,0.91454140826703012,"[0.73041718, 0.18698354]","[0.92884044, 0.59578656]",9,30,40,3,5
28,3.182350e-16,2021-10-04_00-32-42,holy7c24203.rc.fas.harvard.edu,7,0.0071123949776022597,0.094548446005977366,0.93211242821420071,"[0.18245262, 0.22982304]","[0.07957338, 0.54931604]",6,50,30,2,4
60,3.182350e-16,2021-10-04_00-33-02,holy7c24203.rc.fas.harvard.edu,8,0.0075996472493229279,0.060733680055341507,0.9618182793473492,"[0.18816457, 0.40685192]","[0.5633916, 0.16836895]",4,40,50,2,3
24,3.182350e-16,2021-10-04_00-32-48,holy7c24203.rc.fas.harvard.edu,9,0.088331316132474386,0.05857065401447941,0.96775940941054295,"[0.64147684, 0.7256815]","[0.4321078, 0.62824842]",2,20,50,5,5


# 2. SHO

## 2.1 Local results

In [7]:
# Read in the SHO results
sho_results = pd.read_csv("local_results/SHO_results.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results)

In [8]:
# Get all rows for the lowest MSE for each random seed
sho_results.loc[sho_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
4,1.380299e-10,2021-06-11_09-48-49,Blake-Laptop,0,0.084103674192579927,0.069158803053338921,0.99421559971500428,0.91628255655640167,18,20,30,2,2
90,2.577914e-07,2021-06-11_14-47-39,Blake-Laptop,1,0.01864159347110542,0.043508768918248619,0.99725868732691547,0.90936707464648725,9,30,40,3,2
1,2.288413e-08,2021-06-11_09-33-53,Blake-Laptop,2,0.016179931742237884,0.0046606843789535387,0.99671190691077982,0.95013448438768733,18,40,30,4,2
75,2.479653e-10,2021-06-11_14-22-08,Blake-Laptop,3,0.077784195494123839,0.076083892874326833,0.99355100246034711,0.94216361247569758,12,20,40,2,3
63,2.718360e-06,2021-06-11_13-59-08,Blake-Laptop,4,0.060863091506102938,0.017629976055315306,0.99638810390437338,0.93814101186107746,17,40,40,4,4
11,3.303208e-10,2021-06-11_10-14-48,Blake-Laptop,5,0.064444985140396788,0.018871354635987887,0.99217858330824571,0.9437169081111938,20,40,40,4,2
98,3.049682e-02,2021-06-11_15-01-06,Blake-Laptop,6,0.070710250808236758,0.079076829478566896,0.99022683984839566,0.92134647279761439,9,40,20,2,2
39,8.936116e-07,2021-06-11_11-08-28,Blake-Laptop,7,0.038719105426380553,0.039024196869717372,0.99431222507291683,0.95728850293879353,11,20,40,2,4
10,4.189644e-10,2021-06-11_10-07-07,Blake-Laptop,8,0.07087432138484609,0.027801696692425892,0.99868401082604674,0.90839619071931299,19,20,30,3,4
78,4.377555e-08,2021-06-11_14-29-58,Blake-Laptop,9,0.034648486748891844,0.071626359821669289,0.99979469110401509,0.94097656224553006,14,40,20,2,3


## 2.2 FASRC results

### 2.2.1 SGD

In [9]:
# Read in the SHO results
sho_results = pd.read_csv("fasrc_results/ray_tune_sho_1.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results)

In [10]:
# Get all rows for the lowest MSE for each random seed
sho_results.loc[sho_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1921,6.201044e-12,2021-09-21_13-30-52,holy7c24304.rc.fas.harvard.edu,0,0.076153458773735264,0.078152800734734948,0.99943943420124004,0.91253528034961962,6,20,40,2,4
1446,4.369432e-11,2021-09-21_13-20-19,holy7c24304.rc.fas.harvard.edu,1,0.055892655578054569,0.026276354515581687,0.99940918754277674,0.90102881047979577,13,50,50,4,5
1500,5.154119e-10,2021-09-21_13-19-52,holy7c24304.rc.fas.harvard.edu,2,0.082684426631297686,0.02472574028721217,0.99770490871826423,0.91246592029249551,13,50,40,3,4
1645,5.844873e-11,2021-09-21_13-22-07,holy7c24304.rc.fas.harvard.edu,3,0.042188577841949237,0.030510460081279469,0.99833918337145322,0.92845027997731722,12,30,20,3,3
1302,7.507520e-11,2021-09-21_13-14-17,holy7c24304.rc.fas.harvard.edu,4,0.058332885523356345,0.059367522970842836,0.99919343866126786,0.92985120181351577,14,20,40,2,4
658,4.761756e-11,2021-09-21_12-57-53,holy7c24304.rc.fas.harvard.edu,5,0.098522024728793137,0.090082875010126254,0.99884570336440903,0.9053923106950964,9,50,40,2,3
688,7.905600e-11,2021-09-21_12-57-23,holy7c24304.rc.fas.harvard.edu,6,0.031963560753648645,0.028694890826260745,0.99255744089627918,0.92637374932984662,18,30,40,4,2
1288,6.498615e-11,2021-09-21_13-13-56,holy7c24304.rc.fas.harvard.edu,7,0.036807670565397063,0.062291911931972276,0.9981204191639832,0.92939354887083747,9,40,50,3,3
1223,6.973702e-12,2021-09-21_13-14-57,holy7c24304.rc.fas.harvard.edu,8,0.053799390179022126,0.01169331684873854,0.9903575816399991,0.96959745230008898,16,50,50,4,5
1195,5.648706e-12,2021-09-21_13-13-41,holy7c24304.rc.fas.harvard.edu,9,0.026270854297383937,0.026940286769479543,0.99070014176829801,0.93941838732399341,12,50,50,5,5


### 2.2.2 Adam

In [25]:
# Read in the SHO results
sho_results = pd.read_csv("fasrc_results/ray_tune_sho_adam_0.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results, optim="adam")

In [26]:
# Get all rows for the lowest MSE for each random seed
sho_results.loc[sho_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1211,2.700784e-12,2021-10-03_10-50-11,holy7c24202.rc.fas.harvard.edu,0,0.00048861147485534354,0.0059646176842056914,0.97862221657386639,"[0.36317042, 0.75224866]","[0.58413382, 0.45382877]",19,50,40,3,3


In [27]:
sho_results = pd.read_csv("fasrc_results/ray_tune_sho_adam.csv", usecols=np.arange(1,22))
sho_results = get_expanded_df(sho_results, optim="adam")
sho_results.loc[sho_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
100,3.754975e-11,2021-10-04_00-51-06,holy7c24402.rc.fas.harvard.edu,0,0.0596631966011004,0.052711308084911646,0.97673090515008942,"[0.2986419, 0.81607355]","[0.79298811, 0.78714233]",6,30,40,2,3
525,2.414083e-11,2021-10-04_01-03-42,holy7c24402.rc.fas.harvard.edu,1,0.081482013451084176,0.020972895200746178,0.95294270439114248,"[0.15865097, 0.1759868]","[0.92397234, 0.94538982]",17,50,40,4,3
1301,2.000985e-10,2021-10-04_01-22-33,holy7c24402.rc.fas.harvard.edu,2,0.079497165921974283,0.03491452380287291,0.93603151335046308,"[0.48017221, 0.78748983]","[0.7407101, 0.52394735]",14,20,40,4,3
1291,1.377069e-11,2021-10-04_01-23-05,holy7c24402.rc.fas.harvard.edu,3,0.026883169370463558,0.035039045212932593,0.94587748195993748,"[0.30716221, 0.31269132]","[0.25308692, 0.02017877]",20,30,20,3,4
1481,1.151153e-11,2021-10-04_01-28-57,holy7c24402.rc.fas.harvard.edu,4,0.019730582492406418,0.030190897197148844,0.97475698088344165,"[0.38809487, 0.95213816]","[0.50725869, 0.1377562]",14,40,20,4,2
1381,6.049151e-11,2021-10-04_01-25-30,holy7c24402.rc.fas.harvard.edu,5,0.057384705059999597,0.011985199935741427,0.95435734973137831,"[0.25453221, 0.90907967]","[0.62094987, 0.40289959]",19,50,30,2,3
380,2.140888e-11,2021-10-04_00-58-51,holy7c24402.rc.fas.harvard.edu,6,0.035088855656472025,0.027207231798821147,0.91253680827666084,"[0.24387189, 0.59774168]","[0.55464001, 0.28889915]",19,30,50,3,2
1532,7.130555e-11,2021-10-04_01-31-14,holy7c24402.rc.fas.harvard.edu,7,0.0069589833435060927,0.011322316686995454,0.97206854381773034,"[0.07110746, 0.94132838]","[0.11494515, 0.03258803]",16,20,30,3,4
1253,1.368435e-11,2021-10-04_01-21-28,holy7c24402.rc.fas.harvard.edu,8,0.060711032031526529,0.017468124046990455,0.94342461439275993,"[0.19005483, 0.7928569]","[0.11367535, 0.0144229]",16,50,40,4,2
1780,3.132567e-11,2021-10-04_01-35-39,holy7c24402.rc.fas.harvard.edu,9,0.091212475745311813,0.017452264687020023,0.95018516322343305,"[0.03027838, 0.77961905]","[0.32926089, 0.19495461]",15,30,40,2,2


# 3. NLO

## 3.1 Local results

In [11]:
# Read in the NLO results
nlo_results = pd.read_csv("local_results/NLO_results.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results)

In [12]:
# Get all rows for the lowest MSE for each random seed
nlo_results.loc[nlo_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
98,2.614585e-02,2021-07-17_06-42-28,Blake-Laptop,0,0.033363705683096989,0.061962699156226979,0.99448815184727812,0.9266888877660302,11,40,40,4,2
141,1.706401e-02,2021-07-17_10-11-32,Blake-Laptop,1,0.080262611324023389,0.055662296162210967,0.99513962266081468,0.96031667439753654,6,50,50,4,5
191,2.559425e-02,2021-07-17_14-36-18,Blake-Laptop,2,0.055791389454363766,0.019971330997096456,0.99435749780329463,0.96016874890304604,7,40,30,2,3
192,2.125287e-02,2021-07-17_14-48-31,Blake-Laptop,3,0.011040224507315451,0.082327211897342364,0.99110136906842816,0.98706755192417694,10,50,30,2,3
44,6.475934e-08,2021-07-17_01-04-52,Blake-Laptop,4,0.024206158241259251,0.078354220036343306,0.99912672104481026,0.94440055450324201,10,20,50,5,5
99,1.118759e-02,2021-07-17_07-10-48,Blake-Laptop,5,0.094168603269354487,0.082331931638144409,0.99645910546096916,0.98744617163500703,17,20,50,2,4
172,8.189200e-03,2021-07-17_13-00-14,Blake-Laptop,6,0.0086462938564157762,0.042859622524960242,0.99948537437820673,0.91440118131197745,12,20,20,4,2
135,2.776054e-02,2021-07-17_09-31-12,Blake-Laptop,7,0.080860549822475231,0.065193876655689001,0.99403962006406743,0.9767748322674118,7,30,40,2,5
178,1.893046e-02,2021-07-17_13-41-26,Blake-Laptop,8,0.070148749571797481,0.088744878092348517,0.9919254394429996,0.9906087654131166,11,30,50,3,3
122,2.225300e-02,2021-07-17_08-43-20,Blake-Laptop,9,0.018547204669798357,0.063220107235321121,0.9980055406954218,0.94518948334914088,17,50,30,2,2


## 3.2 FASRC results

### 3.2.1 SGD

In [13]:
# Read in the NLO results
nlo_results = pd.read_csv("fasrc_results/ray_tune_nlo_1.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results)

In [14]:
# Get all rows for the lowest MSE for each random seed
nlo_results.loc[nlo_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
193,3.030222e-06,2021-09-22_10-57-08,holy7c24204.rc.fas.harvard.edu,0,0.06491036168359976,0.066507743972948757,0.99955638467483721,0.91331272861034496,17,20,20,2,4
1508,7.539362e-03,2021-09-22_12-13-06,holy7c24204.rc.fas.harvard.edu,1,0.064870015233960382,0.051243752014137875,0.9993762945947593,0.93830720131878931,6,20,50,4,5
973,6.485889e-03,2021-09-22_11-33-29,holy7c24204.rc.fas.harvard.edu,2,0.038298723636900379,0.090130502686449315,0.99967815276871319,0.95713917531628456,12,30,20,4,3
1451,1.128963e-04,2021-09-22_12-07-42,holy7c24204.rc.fas.harvard.edu,3,0.024451629375748584,0.090538818132850751,0.99704383521317552,0.95267877502051879,14,30,40,5,4
420,2.532581e-08,2021-09-22_11-11-32,holy7c24204.rc.fas.harvard.edu,4,0.072468214040095222,0.077998012031143579,0.99973883642742589,0.91185432671605138,5,20,30,3,4
839,9.570328e-03,2021-09-22_11-36-33,holy7c24204.rc.fas.harvard.edu,5,0.00042097963911614101,0.08869410126760717,0.99147870717103248,0.96917177058835546,19,20,40,3,5
58,1.031635e-02,2021-09-22_10-54-59,holy7c24204.rc.fas.harvard.edu,6,0.038525157521168876,0.088079675345949224,0.99699564947265806,0.96328484187184893,20,40,40,2,5
1077,6.691912e-03,2021-09-22_11-47-31,holy7c24204.rc.fas.harvard.edu,7,0.0075360750488506021,0.092818029961952583,0.99696808707965678,0.98153781108290894,11,30,30,5,4
198,2.595064e-03,2021-09-22_11-04-26,holy7c24204.rc.fas.harvard.edu,8,0.080450396448642866,0.072885997934465194,0.99907016060113707,0.95321723696431859,20,30,50,4,5
329,1.672899e-03,2021-09-22_11-09-29,holy7c24204.rc.fas.harvard.edu,9,0.042147470017390803,0.098470404850642013,0.99985213430249109,0.98188281052104154,11,40,50,3,4


### 3.2.1 Adam

In [28]:
# Read in the NLO results
nlo_results = pd.read_csv("fasrc_results/ray_tune_nlo_adam_0.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results, optim="adam")

In [29]:
# Get all rows for the lowest MSE for each random seed
nlo_results.loc[nlo_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1807,7.681183e-13,2021-10-03_12-05-24,holy7c24203.rc.fas.harvard.edu,0,0.02104329548894603,0.010648723263851936,0.98037812391306267,"[0.22544921, 0.33148318]","[0.36273349, 0.55130697]",15,20,40,2,4


In [30]:
nlo_results = pd.read_csv("fasrc_results/ray_tune_nlo_adam.csv", usecols=np.arange(1,22))
nlo_results = get_expanded_df(nlo_results, optim="adam")
nlo_results.loc[nlo_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
284,9.808947e-10,2021-10-04_02-01-49,holy7c24203.rc.fas.harvard.edu,0,0.078757774287542187,0.057909099876832029,0.9630903539432345,"[0.14775712, 0.96484908]","[0.66156042, 0.80956499]",13,30,40,2,2
415,2.477026e-11,2021-10-04_02-15-57,holy7c24203.rc.fas.harvard.edu,1,0.053315184448687492,0.015473814091325765,0.99727513521880695,"[0.12763741, 0.63022683]","[0.63737985, 0.56072805]",5,20,20,2,4
163,2.325411e-11,2021-10-04_02-00-33,holy7c24203.rc.fas.harvard.edu,2,0.038311077314438662,0.015022903009935732,0.99210646762779864,"[0.0007779, 0.4103747]","[0.4396353, 0.97088812]",9,20,30,2,5
1252,2.144644e-11,2021-10-04_02-55-23,holy7c24203.rc.fas.harvard.edu,3,0.065515162353922643,0.0098007846645461315,0.98249102516659292,"[0.7524641, 0.93424283]","[0.85173115, 0.77125918]",18,20,30,2,4
1305,1.177579e-10,2021-10-04_02-52-43,holy7c24203.rc.fas.harvard.edu,4,0.080131021892641818,0.031406284938896807,0.95807223432904531,"[0.0386128, 0.48618842]","[0.37943322, 0.74241767]",8,50,40,3,2
1832,1.405828e-11,2021-10-04_03-22-36,holy7c24203.rc.fas.harvard.edu,5,0.054268860988652641,0.028273693898291843,0.97731931039021869,"[0.34638868, 0.87557239]","[0.32652425, 0.38327061]",17,40,50,2,2
1075,6.206875e-11,2021-10-04_02-42-48,holy7c24203.rc.fas.harvard.edu,6,0.032902816013771354,0.012344431104217121,0.94696271913397401,"[0.22870481, 0.35836412]","[0.7326032, 0.88042452]",19,40,30,4,5
1882,1.105297e-10,2021-10-04_03-22-44,holy7c24203.rc.fas.harvard.edu,7,0.011152825468485551,0.01514217009452553,0.93454392136809183,"[0.06337378, 0.51350646]","[0.53766632, 0.92900947]",15,50,50,2,4
1226,9.346169e-11,2021-10-04_02-56-54,holy7c24203.rc.fas.harvard.edu,8,0.035189586751030809,0.0029219054228305972,0.98873793195803494,"[0.12110837, 0.71524234]","[0.40615609, 0.76406156]",12,20,50,2,5
575,7.861828e-12,2021-10-04_02-24-42,holy7c24203.rc.fas.harvard.edu,9,0.02530252267379016,0.012594526246066931,0.997966413094176,"[0.14728888, 0.83252982]","[0.81857537, 0.40475608]",3,50,30,2,4


# 4. POS

## 4.1 Local results

In [15]:
# Read in the POS results
pos_results = pd.read_csv("local_results/POS_results.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results)

In [16]:
# Get all rows for the lowest MSE for each random seed
pos_results.loc[pos_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
40,6.731801e-10,2021-08-24_14-46-43,Blake-Laptop,0,0.07557494835345345,0.016915835653294964,0.99810123660954464,0.9302252022792793,2,20,40,4,5
13,1.480677e-09,2021-08-24_13-35-49,Blake-Laptop,1,0.045856283274370184,0.063801150325135239,0.99950181858504628,0.90133572596281042,17,40,50,5,2
198,1.743285e-10,2021-08-24_20-23-00,Blake-Laptop,2,2.8048901958568326e-05,0.047090434824697838,0.9995684127767277,0.9125563963435388,20,20,40,4,4
187,5.324107e-11,2021-08-24_19-55-47,Blake-Laptop,3,0.0051482807093043304,0.031988599372662956,0.99888002516943564,0.91363784252025093,5,30,20,5,5
90,9.117905e-10,2021-08-24_16-37-02,Blake-Laptop,4,0.082451273852072196,0.025539200430532285,0.99188526248310305,0.92862447231676071,10,20,40,5,5
119,1.862064e-09,2021-08-24_17-25-19,Blake-Laptop,5,0.014539330717385215,0.031225242795451174,0.99653472685790145,0.92037825166103771,10,30,40,4,4
72,3.208514e-10,2021-08-24_16-04-17,Blake-Laptop,6,0.02092082991298564,0.069687317929642695,0.99645073271240947,0.93850547104646687,7,20,40,3,3
172,2.595747e-10,2021-08-24_19-20-51,Blake-Laptop,7,0.091194956504458885,0.050306122678116003,0.99635910025038166,0.90056746159481571,9,40,50,5,3
116,1.519133e-08,2021-08-24_17-15-15,Blake-Laptop,8,0.076766430816860187,0.061589557751890345,0.99721235512349593,0.94057098496206881,17,20,30,4,3
54,1.745204e-09,2021-08-24_15-15-34,Blake-Laptop,9,0.073977994359567437,0.084630461037161347,0.99930888967031639,0.90507427671467788,8,30,20,3,2


## 4.2 FASRC results

### 4.2.1 SGD

In [17]:
# Read in the POS results
pos_results = pd.read_csv("fasrc_results/ray_tune_pos_1.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results)

In [18]:
# Get all rows for the lowest MSE for each random seed
pos_results.loc[pos_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1627,2.225172e-11,2021-09-22_12-13-20,holy7c24402.rc.fas.harvard.edu,0,0.098221150399786114,0.066607573411165688,0.99894345328173861,0.91551577798080352,18,20,40,4,4
775,1.615727e-10,2021-09-22_11-55-15,holy7c24402.rc.fas.harvard.edu,1,0.07107250680186615,0.081335450339348109,0.99974250573138423,0.91515487986456945,7,40,30,4,2
669,9.134057e-12,2021-09-22_11-53-15,holy7c24402.rc.fas.harvard.edu,2,0.055806632084829388,0.048591723785277229,0.99379113071435632,0.91277226078365592,11,20,20,4,5
33,1.286095e-11,2021-09-22_11-39-38,holy7c24402.rc.fas.harvard.edu,3,0.08916741249614199,0.029412794594915106,0.99111954165814753,0.91563552619432476,19,20,20,5,2
582,1.034194e-11,2021-09-22_11-52-11,holy7c24402.rc.fas.harvard.edu,4,0.0084192199742006545,0.033407060812362492,0.99151313161450583,0.93566039921920963,14,20,30,5,5
169,2.149393e-10,2021-09-22_11-42-35,holy7c24402.rc.fas.harvard.edu,5,0.033821536519924027,0.080026963269880458,0.99014014470868339,0.93250772026528572,20,50,30,3,2
156,1.818886e-10,2021-09-22_11-41-30,holy7c24402.rc.fas.harvard.edu,6,0.04583947129816443,0.01597957281781821,0.99793703118152,0.94443996484001458,20,30,20,4,2
1791,2.814301e-10,2021-09-22_12-15-34,holy7c24402.rc.fas.harvard.edu,7,0.021322676303514594,0.015344585212026262,0.99732869396399804,0.95541907301728202,19,20,30,5,3
1873,2.210848e-10,2021-09-22_12-18-21,holy7c24402.rc.fas.harvard.edu,8,0.099730955988570208,0.094093860843772789,0.99765918327277292,0.93965811805123312,6,40,30,4,5
97,1.327367e-10,2021-09-22_11-42-11,holy7c24402.rc.fas.harvard.edu,9,0.050908229764477994,0.096421065561658401,0.99382984398319374,0.92111264232100132,10,40,20,5,5


### 4.2.2 Adam

In [31]:
# Read in the POS results
pos_results = pd.read_csv("fasrc_results/ray_tune_pos_adam_0.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results, optim="adam")

In [32]:
# Get all rows for the lowest MSE for each random seed
pos_results.loc[pos_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
938,6.398321e-13,2021-10-03_11-33-14,holy7c24201.rc.fas.harvard.edu,0,0.02154824129565415,0.019086634787137204,0.95727502453840718,"[0.13931808, 0.36966975]","[0.74574617, 0.7590883]",9,30,50,2,4


In [33]:
pos_results = pd.read_csv("fasrc_results/ray_tune_pos_adam.csv", usecols=np.arange(1,22))
pos_results = get_expanded_df(pos_results, optim="adam")
pos_results.loc[pos_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
997,2.674505e-12,2021-10-04_03-59-30,holy7c24204.rc.fas.harvard.edu,0,0.002862002287040295,0.039911848889371215,0.91814314929064222,"[0.44946785, 0.67016223]","[0.31598038, 0.05736258]",13,50,50,4,2
548,6.910650e-12,2021-10-04_03-51-23,holy7c24204.rc.fas.harvard.edu,1,0.057298925024592481,0.016586868864287833,0.90357097426250998,"[0.03806089, 0.03315012]","[0.33871399, 0.93239932]",19,30,50,2,4
415,1.697799e-12,2021-10-04_03-47-52,holy7c24204.rc.fas.harvard.edu,2,0.011109899699203585,0.051963063181141911,0.90549502145300442,"[0.15813326, 0.58541457]","[0.75701359, 0.92306149]",18,30,20,4,3
1992,4.065137e-12,2021-10-04_04-18-16,holy7c24204.rc.fas.harvard.edu,3,0.032343791399860142,0.050289954260823534,0.93761463063234418,"[0.11490233, 0.61101461]","[0.04168844, 0.81135499]",11,30,50,3,2
1157,2.022413e-12,2021-10-04_04-02-42,holy7c24204.rc.fas.harvard.edu,4,0.0022530499249221146,0.015233439230105537,0.92352631080597347,"[0.21238117, 0.34101081]","[0.72426908, 0.83563404]",16,50,50,2,3
1263,1.550480e-11,2021-10-04_04-03-16,holy7c24204.rc.fas.harvard.edu,5,0.022803476933138247,0.058329290830044164,0.93746838790134512,"[0.06428135, 0.57210522]","[0.62831873, 0.06302207]",9,20,20,2,4
1675,2.336244e-12,2021-10-04_04-13-11,holy7c24204.rc.fas.harvard.edu,6,0.0020348054417380001,0.046377606361762747,0.91589635730793684,"[0.24690103, 0.62241883]","[0.84348577, 0.59985862]",18,50,30,5,3
1698,2.374962e-12,2021-10-04_04-12-59,holy7c24204.rc.fas.harvard.edu,7,0.0040492191793375506,0.010205898712660437,0.94943630323069073,"[0.21645442, 0.3894066]","[0.47660815, 0.64173763]",12,30,40,4,2
1258,4.995612e-12,2021-10-04_04-06-40,holy7c24204.rc.fas.harvard.edu,8,0.025378810568192106,0.0067115008631685701,0.95936631558598862,"[0.58416337, 0.8549995]","[0.88298712, 0.96570292]",12,20,50,4,4
41,8.835623e-12,2021-10-04_03-41-33,holy7c24204.rc.fas.harvard.edu,9,0.09336123638994899,0.0075975444363885194,0.90967028498051161,"[0.08499967, 0.93407971]","[0.19908629, 0.5786902]",18,40,50,2,4


# 5. SIR

## 5.1 Local results

In [19]:
# Read in the SIR results
sir_results = pd.read_csv("local_results/SIR_results_2.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results)

In [20]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
144,0.007679,2021-08-30_16-22-11,Blake-Laptop,0,0.063245058081627822,0.0030873557016016604,0.99821762927590441,0.91196329101448315,12,30,30,3,3
175,0.039644,2021-08-30_21-28-18,Blake-Laptop,1,0.022559762927580863,0.013985917575822997,0.99364718317733325,0.96924577825550007,12,30,40,2,2
191,0.028068,2021-08-30_23-17-00,Blake-Laptop,2,0.025016684568462212,0.036543204025094318,0.9986853939756587,0.91245316297868073,13,50,20,2,4
14,0.044576,2021-08-29_18-07-40,Blake-Laptop,3,0.095522415603877767,0.0023482387302428999,0.9927647470528107,0.97920269412832794,18,40,30,2,4
64,0.013378,2021-08-30_00-29-54,Blake-Laptop,4,0.011940041955991321,0.0032497864640433204,0.99696830694153227,0.92170723417628164,3,20,40,5,3
124,0.009868,2021-08-30_12-54-06,Blake-Laptop,5,0.018335756901101161,0.0030162341652158007,0.99429154307988421,0.92692343661824683,6,20,30,5,4
84,0.029765,2021-08-30_06-25-12,Blake-Laptop,6,0.090712599353006032,0.013418818100989943,0.99820726078731892,0.93885194799252369,13,50,40,2,4
40,0.008703,2021-08-29_21-33-30,Blake-Laptop,7,0.087183848322769833,0.0075214154361043931,0.99638570338713983,0.90436198329629358,13,40,20,3,2
164,0.398470,2021-08-30_19-10-00,Blake-Laptop,8,0.048215734161919425,0.017189105189115282,0.99113159412563245,0.94442028232404607,11,40,40,2,4
10,0.009820,2021-08-29_17-06-38,Blake-Laptop,9,0.021558935786255971,0.0035669182692668658,0.99330088171266429,0.91639654254592229,8,20,50,3,4


## 5.2 FASRC results

### 5.2.1 Normal networks

In [21]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_2.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results)

In [22]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1851,0.006339,2021-09-23_11-28-48,holy7c24101.rc.fas.harvard.edu,0,0.92208380930790046,0.26597518069713805,0.85723857335899367,0.049479452839425273,17,30,40,4,5
315,0.039040,2021-09-23_09-32-01,holy7c24101.rc.fas.harvard.edu,1,0.81871467557476008,0.3879290174235771,0.57931210116696474,0.094183982477327205,13,20,30,3,4
1289,0.036784,2021-09-23_10-45-15,holy7c24101.rc.fas.harvard.edu,2,0.022318127863631739,0.12303184561904569,0.7195562967374558,0.35222762907354149,11,30,40,4,5
1050,0.021509,2021-09-23_10-25-46,holy7c24101.rc.fas.harvard.edu,3,0.36773623130938388,0.19583389929732309,0.82337468756448329,0.59902524540108315,14,50,20,5,2
1253,0.018376,2021-09-23_10-42-30,holy7c24101.rc.fas.harvard.edu,4,0.84965386722142999,0.035440430813887487,0.70993697459044258,0.59248623727192096,19,20,40,3,4
1185,0.053411,2021-09-23_10-36-38,holy7c24101.rc.fas.harvard.edu,5,0.90829941898920485,0.36601183447757701,0.81874049310464714,0.14943017030284655,10,30,20,3,4
1521,0.039716,2021-09-23_11-02-53,holy7c24101.rc.fas.harvard.edu,6,0.78001151834558891,0.47633881529793742,0.8270020648894193,0.74290011166262515,10,50,20,3,2
1667,0.041591,2021-09-23_11-13-37,holy7c24101.rc.fas.harvard.edu,7,0.054796183307596363,0.20213812879720297,0.41162058779440847,0.20638426814018104,15,20,40,3,2
1984,0.013095,2021-09-23_11-39-25,holy7c24101.rc.fas.harvard.edu,8,0.84148827460913012,0.10595653947133063,0.85671881348766565,0.4625621932561147,15,20,30,5,5
8,0.016060,2021-09-23_09-12-38,holy7c24101.rc.fas.harvard.edu,9,0.094457031795990853,0.24144835280463828,0.7103902519342471,0.50158621885726873,19,40,50,5,3


### 5.2.2. Deeper networks

Tuned with number of layers bound by [6, 7, 8, 9].

In [23]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_deep.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results)

In [24]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1891,0.007247,2021-09-21_22-54-36,holy7c24404.rc.fas.harvard.edu,0,0.071146004080405389,0.0030459077806545293,0.99449054521696256,0.91137865766388138,11,40,30,6,9
649,0.006928,2021-09-21_20-08-40,holy7c24404.rc.fas.harvard.edu,1,0.015554531264675068,0.015416533534517499,0.99148992959061999,0.90764444924672461,4,20,20,7,8
888,0.007737,2021-09-21_20-47-45,holy7c24404.rc.fas.harvard.edu,2,0.023010916396903391,0.0032655723145411843,0.9901794525350347,0.92182357429473694,6,40,20,7,8
1754,0.014215,2021-09-21_22-40-38,holy7c24404.rc.fas.harvard.edu,3,0.041732580580658145,0.0001143222817996133,0.99742785230539288,0.91994964416171632,17,40,50,6,9
47,0.015275,2021-09-21_18-58-13,holy7c24404.rc.fas.harvard.edu,4,0.043018853694825006,0.0059103421933710317,0.99782771620027499,0.90445381368987476,3,30,30,7,8
1833,0.007725,2021-09-21_23-15-03,holy7c24404.rc.fas.harvard.edu,5,0.0053861739977008423,0.00082212632114735389,0.99931540800705498,0.90018518121999269,7,30,50,6,8
1360,0.022560,2021-09-21_21-39-51,holy7c24404.rc.fas.harvard.edu,6,0.024425952210704784,0.077991177559394192,0.99317975166901051,0.90119488598524433,19,50,40,7,8
1948,0.025213,2021-09-21_23-04-23,holy7c24404.rc.fas.harvard.edu,7,0.022146057664326722,0.00091419870210874947,0.99213541611915346,0.9087689170740596,8,50,30,9,6
1700,0.004459,2021-09-21_22-30-04,holy7c24404.rc.fas.harvard.edu,8,0.019466982693652577,0.0022531776388646035,0.99737711628233727,0.94114125988931008,5,20,30,7,6
1729,0.006036,2021-09-21_22-33-53,holy7c24404.rc.fas.harvard.edu,9,0.0070188752405563864,0.0033947851657198675,0.99818261383892481,0.9212750838315118,3,20,40,6,9


### 5.2.3. Bigger networks

Tuned with number of layers bound by [6, 7, 8, 9] and number of nodes bound by [60, 70, 80, 90].

In [5]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_big.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, separate_momentum=True)

In [6]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,d_momentum,g_momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
754,0.005873,2021-10-01_14-33-07,holy7c24301.rc.fas.harvard.edu,0,0.044363902885368914,0.026437478782035659,0.99295286669429272,0.087965455326998579,0.539780,19,90,70,9,9
753,0.006748,2021-10-01_13-41-10,holy7c24301.rc.fas.harvard.edu,1,0.074113912995316972,0.072953179561030243,0.98386539173204435,0.68367304369492965,0.606575,12,70,60,8,7
701,0.008500,2021-10-01_14-02-32,holy7c24301.rc.fas.harvard.edu,2,0.0062633670349174437,0.033418850430215155,0.99410754583212391,0.59693141481321554,0.795930,18,80,60,8,6
948,0.019542,2021-10-01_14-34-15,holy7c24301.rc.fas.harvard.edu,3,0.016622900460257765,0.024959808872723914,0.96124328881262755,0.10069924390296872,0.759561,17,80,90,8,6
641,0.013406,2021-10-01_13-13-47,holy7c24301.rc.fas.harvard.edu,4,0.042517417075091775,0.040191813413940423,0.95966610267852248,0.5086605585329963,0.274348,17,60,60,8,7
452,0.011417,2021-10-01_12-29-40,holy7c24301.rc.fas.harvard.edu,5,0.090697635938595964,0.01632291771611433,0.98418636315805175,0.041560979487818957,0.686447,11,70,60,7,7
91,0.011128,2021-10-01_11-12-25,holy7c24301.rc.fas.harvard.edu,6,0.046324376416375067,0.048763449080906775,0.99504556504077202,0.84198167345274577,0.286498,20,90,90,6,8
714,0.025067,2021-10-01_13-30-27,holy7c24301.rc.fas.harvard.edu,7,0.090317888024000093,0.014546219307318472,0.94906425177544051,0.50101332809570953,0.653254,13,90,70,6,8
725,0.011583,2021-10-01_13-35-01,holy7c24301.rc.fas.harvard.edu,8,0.011486904058870732,0.06122099925523708,0.98169852643775068,0.58158081585221477,0.605684,17,60,60,6,6
496,0.034302,2021-10-01_12-36-26,holy7c24301.rc.fas.harvard.edu,9,0.030764949680190279,0.072805452296024406,0.89639883541203558,0.4364514113186298,0.032743,20,90,60,8,7


### 5.2.4 Higher learning rates

Tuned with learning rates bound by [1e-4, 1.].

In [29]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_lr.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results)

In [30]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
1851,0.006339,2021-09-23_11-28-48,holy7c24101.rc.fas.harvard.edu,0,0.92208380930790046,0.26597518069713805,0.85723857335899367,0.049479452839425273,17,30,40,4,5
315,0.039040,2021-09-23_09-32-01,holy7c24101.rc.fas.harvard.edu,1,0.81871467557476008,0.3879290174235771,0.57931210116696474,0.094183982477327205,13,20,30,3,4
1289,0.036784,2021-09-23_10-45-15,holy7c24101.rc.fas.harvard.edu,2,0.022318127863631739,0.12303184561904569,0.7195562967374558,0.35222762907354149,11,30,40,4,5
1050,0.021509,2021-09-23_10-25-46,holy7c24101.rc.fas.harvard.edu,3,0.36773623130938388,0.19583389929732309,0.82337468756448329,0.59902524540108315,14,50,20,5,2
1253,0.018376,2021-09-23_10-42-30,holy7c24101.rc.fas.harvard.edu,4,0.84965386722142999,0.035440430813887487,0.70993697459044258,0.59248623727192096,19,20,40,3,4
1185,0.053411,2021-09-23_10-36-38,holy7c24101.rc.fas.harvard.edu,5,0.90829941898920485,0.36601183447757701,0.81874049310464714,0.14943017030284655,10,30,20,3,4
1521,0.039716,2021-09-23_11-02-53,holy7c24101.rc.fas.harvard.edu,6,0.78001151834558891,0.47633881529793742,0.8270020648894193,0.74290011166262515,10,50,20,3,2
1667,0.041591,2021-09-23_11-13-37,holy7c24101.rc.fas.harvard.edu,7,0.054796183307596363,0.20213812879720297,0.41162058779440847,0.20638426814018104,15,20,40,3,2
1984,0.013095,2021-09-23_11-39-25,holy7c24101.rc.fas.harvard.edu,8,0.84148827460913012,0.10595653947133063,0.85671881348766565,0.4625621932561147,15,20,30,5,5
8,0.016060,2021-09-23_09-12-38,holy7c24101.rc.fas.harvard.edu,9,0.094457031795990853,0.24144835280463828,0.7103902519342471,0.50158621885726873,19,40,50,5,3


### 5.2.5 Reduced gamma and momentum

Tuned with gamma bound by [0.09, 0.999] and momentum bound by [0.009, 0.99].

In [31]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_momentum.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results)

In [32]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
556,0.023502,2021-09-22_23-14-37,holy7c24102.rc.fas.harvard.edu,0,0.080730346208676335,0.049108548946200786,0.91351966622586434,0.72700921829027887,16,20,40,4,3
1437,0.009164,2021-09-23_00-30-25,holy7c24102.rc.fas.harvard.edu,1,0.069697612674551032,0.012481411659236897,0.98049349535235497,0.80991758509265555,11,30,40,3,5
1545,0.011008,2021-09-23_00-39-03,holy7c24102.rc.fas.harvard.edu,2,0.069206409166307936,0.077395752162159978,0.98105478994997053,0.77677597789793396,6,30,20,3,4
1052,0.010945,2021-09-22_23-56-46,holy7c24102.rc.fas.harvard.edu,3,0.095052450725840051,0.066838895217096558,0.97024474796134741,0.51057137073094072,15,20,40,4,5
50,0.005627,2021-09-22_22-37-44,holy7c24102.rc.fas.harvard.edu,4,0.076460646089302572,0.001566270353284415,0.98964647369986658,0.84364032929023192,18,40,30,5,3
99,0.013772,2021-09-22_22-39-24,holy7c24102.rc.fas.harvard.edu,5,0.073049182897777523,0.064439060222837727,0.94511772697111218,0.67321138497032174,12,30,20,5,4
917,0.007503,2021-09-22_23-45-44,holy7c24102.rc.fas.harvard.edu,6,0.057828526151014151,0.012261576863363178,0.98465402062732954,0.56422147146534418,19,50,30,4,2
622,0.016741,2021-09-22_23-19-27,holy7c24102.rc.fas.harvard.edu,7,0.021103283829890577,0.055621335455408406,0.97438658675904322,0.13087093699014682,17,20,20,4,2
1519,0.014690,2021-09-23_00-38-06,holy7c24102.rc.fas.harvard.edu,8,0.083442548904807942,0.085186761973212885,0.98803665581694733,0.34560462552106153,14,40,20,4,4
1938,0.012160,2021-09-23_01-09-58,holy7c24102.rc.fas.harvard.edu,9,0.039480323096511089,0.094431542576873376,0.78470206740286486,0.54624836527286391,15,30,40,3,5


### 5.2.6 Separate momentum for G and D

Tuned with gamma bound by [0.1, 0.999] and the momentum of G and D (separate hyperparameters!) bound by [0, 0.99].

In [38]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_two_momentum.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, separate_momentum=True)

In [39]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,d_momentum,g_momentum,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
256,0.020852,2021-09-29_15-13-23,holy7c24304.rc.fas.harvard.edu,0,0.091711683148532785,0.039294697167586441,0.92118676522695997,0.57866176785428214,0.710130,19,30,50,2,4
769,0.017482,2021-09-29_15-54-01,holy7c24304.rc.fas.harvard.edu,1,0.0045619397670406559,0.083075130257360866,0.98726280896723617,0.73202091348416987,0.105935,8,30,20,4,2
791,0.017158,2021-09-29_15-55-24,holy7c24304.rc.fas.harvard.edu,2,0.049122902880199035,0.074089954731326971,0.95840508313649653,0.056007497937579052,0.220288,8,40,20,5,2
187,0.013320,2021-09-29_15-09-07,holy7c24304.rc.fas.harvard.edu,3,0.098067099054594958,0.020663153801233881,0.95021939919757581,0.72582088045722282,0.697785,13,40,40,5,3
229,0.008587,2021-09-29_15-13-21,holy7c24304.rc.fas.harvard.edu,4,0.052941616993703318,0.094251293698117869,0.9965860010287213,0.77579460143843149,0.157321,5,40,40,3,4
890,0.014425,2021-09-29_16-03-58,holy7c24304.rc.fas.harvard.edu,5,0.00084525040340476388,0.089033709164392322,0.99298464339426118,0.32952465136450348,0.268229,12,20,30,4,4
459,0.043567,2021-09-29_15-28-57,holy7c24304.rc.fas.harvard.edu,6,0.039459023395400299,0.083533840840597048,0.37596487335750495,0.80079796895761346,0.632478,15,20,50,5,3
671,0.022549,2021-09-29_15-46-47,holy7c24304.rc.fas.harvard.edu,7,0.036729194842129533,0.096186882246887637,0.70587917203978445,0.61890224854450693,0.175311,19,40,40,4,5
491,0.012413,2021-09-29_15-33-32,holy7c24304.rc.fas.harvard.edu,8,0.028242740028702036,0.084728416316747601,0.9580131693713565,0.88616512425523331,0.330043,11,30,50,4,3
830,0.024309,2021-09-29_15-58-02,holy7c24304.rc.fas.harvard.edu,9,0.040316121501683254,0.059403559863857987,0.84419071618660813,0.32487089336833075,0.691971,20,40,40,4,5


### 5.2.7 Adam optimizer

To reproduce Dylan's results, we tried hyperparameter tuning with the old Adam optimizer. The results are much better.

In [25]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/ray_tune_sir_adam.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, optim="adam")

In [26]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
635,7.945735e-08,2021-09-28_14-15-03,holy7c24301.rc.fas.harvard.edu,0,0.027026421938530359,0.038459593925159519,0.99042958052411334,"[0.12369985, 0.95286534]","[0.65024054, 0.78465781]",12,40,50,5,2
1426,6.062324e-09,2021-09-28_15-26-42,holy7c24301.rc.fas.harvard.edu,1,0.025588658112758851,0.012579516410734513,0.99814984932463779,"[0.33433906, 0.85682265]","[0.42760688, 0.69871359]",4,40,40,4,2
64,1.686890e-07,2021-09-28_13-37-01,holy7c24301.rc.fas.harvard.edu,2,0.07756967731524167,0.02212446132843747,0.9962818994070447,"[0.13688223, 0.24808193]","[0.42253514, 0.58008496]",13,20,20,2,3
990,1.993337e-09,2021-09-28_14-46-37,holy7c24301.rc.fas.harvard.edu,3,0.048353644841351323,0.033969438603135428,0.99402277349277524,"[0.04489709, 0.20844875]","[0.16923791, 0.31905255]",15,50,20,2,2
487,2.206559e-05,2021-09-28_14-07-59,holy7c24301.rc.fas.harvard.edu,4,0.040588551284938872,0.023947688905797238,0.99719180130571694,"[0.23898053, 0.6159229]","[0.7328925, 0.72766545]",13,40,20,3,2
1050,2.176410e-08,2021-09-28_14-54-15,holy7c24301.rc.fas.harvard.edu,5,0.05582854527858086,0.0090515126509902941,0.99423743904864825,"[0.045401, 0.13825048]","[0.36934164, 0.47795483]",19,50,30,3,3
1834,1.777741e-08,2021-09-28_16-00-13,holy7c24301.rc.fas.harvard.edu,6,0.0073991874304079261,0.014385527752860105,0.99287859304381532,"[0.11268679, 0.72244327]","[0.48248225, 0.27331328]",10,30,50,2,4
174,1.422065e-08,2021-09-28_13-43-55,holy7c24301.rc.fas.harvard.edu,7,0.05579851320138457,0.057942752795427162,0.99148349671655833,"[0.1048081, 0.88840105]","[0.32783042, 0.33873572]",14,40,20,2,2
404,3.263880e-06,2021-09-28_14-04-32,holy7c24301.rc.fas.harvard.edu,8,0.056473341747855975,0.031625885712984962,0.99226237212549739,"[0.04374815, 0.83982155]","[0.09092112, 0.27708352]",11,50,30,3,3
1296,7.686027e-03,2021-09-28_15-16-32,holy7c24301.rc.fas.harvard.edu,9,0.066487417794533515,0.01364862542592492,0.99296972960754593,"[0.36749743, 0.201053]","[0.06913367, 0.46259216]",18,50,50,3,2


### 5.2.8 Adam optimizer (tuning with noise)

In [27]:
# Read in the SIR results
sir_results = pd.read_csv("fasrc_results/noise/ray_tune_sir_adam_3.csv", usecols=np.arange(1,22))
sir_results = get_expanded_df(sir_results, optim="adam")

In [28]:
# Get all rows for the lowest MSE for each random seed
sir_results.loc[sir_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
305,8.094921e-10,2021-09-29_02-05-44,holy7c24301.rc.fas.harvard.edu,0,0.002436298633057405,0.009867835819474308,0.99639513528613899,"[0.22477554, 0.42179324]","[0.24709502, 0.99214329]",12,20,20,5,5
1946,6.429450e-09,2021-09-29_04-10-42,holy7c24301.rc.fas.harvard.edu,1,0.053070623877773823,0.012613704240178837,0.99283464798131915,"[0.32836557, 0.6167217]","[0.04552585, 0.9023211]",20,20,40,4,2
498,5.425772e-08,2021-09-29_02-15-42,holy7c24301.rc.fas.harvard.edu,2,0.098578987246042568,0.029642011499673843,0.99319253837793853,"[0.1544031, 0.14338693]","[0.04722817, 0.44114811]",16,20,20,4,2
713,3.815878e-05,2021-09-29_02-33-28,holy7c24301.rc.fas.harvard.edu,3,0.012266532564652832,0.00086762436655861896,0.99835925979073115,"[0.5237571, 0.73918804]","[0.10709423, 0.07448841]",8,50,40,2,2
609,2.067624e-08,2021-09-29_02-14-06,holy7c24301.rc.fas.harvard.edu,4,0.033038376827998303,0.029583563066477751,0.99429449084881472,"[0.18401837, 0.67788482]","[0.14907621, 0.67257946]",3,20,30,4,2
930,4.733850e-08,2021-09-29_02-48-06,holy7c24301.rc.fas.harvard.edu,5,0.0091274578180204381,0.017928587985089846,0.99461928816881084,"[0.10459049, 0.83887993]","[0.21123357, 0.24090213]",18,20,20,2,2
1333,2.239005e-08,2021-09-29_03-25-13,holy7c24301.rc.fas.harvard.edu,6,0.023548824744149516,0.016405108042785502,0.9942906452372634,"[0.07720514, 0.37868219]","[0.21724342, 0.90840084]",15,20,40,4,5
89,5.450266e-10,2021-09-29_01-40-30,holy7c24301.rc.fas.harvard.edu,7,0.020265461563507874,0.010975074883327404,0.995944400264417,"[0.09947991, 0.49056625]","[0.28996158, 0.72606659]",10,50,30,2,2
1679,4.194547e-10,2021-09-29_03-49-49,holy7c24301.rc.fas.harvard.edu,8,0.036939741534521356,0.016964876067420524,0.99116320637897382,"[0.03188197, 0.33825203]","[0.10605551, 0.91332452]",14,30,50,2,5
2,9.449197e-05,2021-09-29_01-36-05,holy7c24301.rc.fas.harvard.edu,9,0.006294346267445643,0.038880928601892475,0.99472129053250036,"[0.03364356, 0.88726782]","[0.80855489, 0.05701105]",6,20,30,3,2


# 6. RANS

## 6.1 FASRC Results

### 6.1.1 Adam optimizer

In [35]:
# Read in the RANS results
rans_results = pd.read_csv("fasrc_results/ray_tune_rans_adam.csv", usecols=np.arange(1,22))
rans_results = get_expanded_df(rans_results, optim="adam")

In [36]:
# Get all rows for the lowest MSE for each random seed
rans_results.loc[rans_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
12,159.826357,2021-10-04_12-29-33,holy7c24401.rc.fas.harvard.edu,0,0.047102637906597022,0.087290889611672293,0.95755480141023575,"[0.35706326, 0.45743051]","[0.05630037, 0.96237312]",2,20,40,4,2
49,162.921950,2021-10-04_12-40-28,holy7c24401.rc.fas.harvard.edu,1,0.013037888418553361,0.068179929128487693,0.91722048060475747,"[0.87877983, 0.74602319]","[0.06234685, 0.63103163]",9,50,20,3,3
51,161.607413,2021-10-04_12-40-14,holy7c24401.rc.fas.harvard.edu,2,0.077748558432677459,0.076515395502946462,0.92417279919135242,"[0.63717351, 0.35698444]","[0.29409961, 0.23383043]",13,40,50,3,3
13,156.151671,2021-10-04_12-30-14,holy7c24401.rc.fas.harvard.edu,3,0.058961726163355238,0.028452255507075089,0.95779376188963516,"[0.7654043, 0.64097131]","[0.83354928, 0.65191265]",12,40,50,5,2
144,158.580074,2021-10-04_13-37-28,holy7c24401.rc.fas.harvard.edu,4,0.054393571770575959,0.053718724748984169,0.94497176541005778,"[0.38566445, 0.95113745]","[0.36412546, 0.21496849]",3,40,20,3,3
68,124.108505,2021-10-04_12-42-06,holy7c24401.rc.fas.harvard.edu,5,0.038028006378184487,0.047213898930652975,0.92032519391415613,"[0.38403368, 0.43670135]","[0.96234936, 0.1028605]",20,30,50,5,3
114,138.253638,2021-10-04_13-07-50,holy7c24401.rc.fas.harvard.edu,6,0.048057269120813588,0.026402378704027226,0.95766592664024797,"[0.15207008, 0.87110651]","[0.88432302, 0.14975234]",9,40,30,3,5
66,160.587602,2021-10-04_12-42-44,holy7c24401.rc.fas.harvard.edu,7,0.094486036958194075,0.086274070812989545,0.93404764420015407,"[0.37594888, 0.52460457]","[0.88219594, 0.80796156]",15,40,30,3,4
176,161.915595,2021-10-04_13-40-39,holy7c24401.rc.fas.harvard.edu,8,0.033656041083068376,0.085933815423378759,0.91017047089719494,"[0.03062641, 0.76398511]","[0.22846284, 0.72158713]",6,50,30,3,2
151,153.241244,2021-10-04_13-29-11,holy7c24401.rc.fas.harvard.edu,9,0.070431661176774449,0.079383774321807182,0.98121480812943707,"[0.06006626, 0.90937366]","[0.36657204, 0.77572945]",13,30,40,4,2


### 6.1.2 Adam optimizer (increased step sizes)

Tuning with `step_size` bounds increased from [2,21] to [180,200].

In [37]:
rans_results = pd.read_csv("fasrc_results/ray_tune_rans_step_180.csv", usecols=np.arange(1,22))
rans_results = get_expanded_df(rans_results, optim="adam")
rans_results.loc[rans_results.groupby("seed")["mean_squared_error"].idxmin()]

,mean_squared_error,date,hostname,seed,d_lr,g_lr,gamma,g_betas,d_betas,step_size,d_n_hidden_units,g_n_hidden_units,d_n_hidden_layers,g_n_hidden_layers
167,148.332530,2021-10-05_00-22-59,holy7c24404.rc.fas.harvard.edu,0,0.099339814820519068,0.091419815971967591,0.91619276603094846,"[0.06794376, 0.87751384]","[0.35354866, 0.81849523]",187,40,30,5,3
105,106.002148,2021-10-04_23-47-31,holy7c24404.rc.fas.harvard.edu,1,0.013129078076143974,0.083953884162904516,0.90830868483317484,"[0.4536513, 0.84105801]","[0.82558762, 0.21714504]",188,30,30,5,2
88,161.302927,2021-10-04_23-43-05,holy7c24404.rc.fas.harvard.edu,2,0.07870666012900962,0.012728947326541201,0.93445092005092178,"[0.75523683, 0.16254932]","[0.91141681, 0.34991591]",198,20,40,3,5
72,149.382446,2021-10-04_23-33-27,holy7c24404.rc.fas.harvard.edu,3,0.059452251898617331,0.043331910468857981,0.95421956919249185,"[0.05469262, 0.93214652]","[0.76317734, 0.22930341]",189,30,40,2,4
21,147.224971,2021-10-04_23-26-21,holy7c24404.rc.fas.harvard.edu,4,0.016437031559906733,0.050577965339591746,0.99389608047800071,"[0.55490866, 0.8822928]","[0.53158871, 0.8049228]",189,30,30,4,4
26,161.959063,2021-10-04_23-20-21,holy7c24404.rc.fas.harvard.edu,5,0.0079000518254977118,0.063548373283614457,0.97107516067680721,"[0.21127108, 0.812236]","[0.55856429, 0.9270129]",182,50,40,5,2
58,156.943443,2021-10-04_23-33-01,holy7c24404.rc.fas.harvard.edu,6,0.0062620415687378523,0.039506738683275582,0.93704891996177608,"[0.36411012, 0.54143152]","[0.0329338, 0.12806263]",195,40,20,3,5
60,116.923019,2021-10-04_23-41-39,holy7c24404.rc.fas.harvard.edu,7,0.032825176378964857,0.022129222306069073,0.9163184698059601,"[0.33675016, 0.58870395]","[0.88221207, 0.66547313]",180,50,30,4,2
0,100.698494,2021-10-04_23-20-16,holy7c24404.rc.fas.harvard.edu,8,0.078454964937072783,0.077056166725618339,0.90982426867466115,"[0.50782583, 0.6315784]","[0.7994576, 0.10848767]",195,40,30,4,2
67,147.999123,2021-10-04_23-32-47,holy7c24404.rc.fas.harvard.edu,9,0.069753793504486308,0.042105229228086224,0.94516660205002656,"[0.89798816, 0.54464049]","[0.51648079, 0.17254799]",181,50,20,5,4
